In [2]:
# Needed only on Google Colab
try:
    import google.colab
    !pip install aiohttp requests zarr
    print('Packages on Google Colab are installed')
except:
    print('Not running on Google Colab')

Not running on Google Colab


In [3]:
import fsspec
import xarray as xr

# Data is located on Green cluster (NYU) and shared with Globus.
* Every individual dataset is a `.zarr` archive [link](https://zarr.readthedocs.io/en/stable/)
* Structure of the folders is the following:
├── eddy
│   ├── 48
│   │   ├── gauss.zarr
│   │   ├── hires-gauss.zarr
│   │   ├── hires-sharp.zarr
│   │   ├── lores.zarr
│   │   └── sharp.zarr
│   ├── 64
│   │   ├── gauss.zarr
│   │   ├── hires-gauss.zarr
│   │   ├── hires-sharp.zarr
│   │   ├── lores.zarr
│   │   └── sharp.zarr
│   ├── 96
│   │   ├── gauss.zarr
│   │   ├── hires-gauss.zarr
│   │   ├── hires-sharp.zarr
│   │   ├── lores.zarr
│   │   └── sharp.zarr
│   └── hires.zarr
├── jet
│   ├── 48
│   │   ├── gauss.zarr
│   │   ├── hires-gauss.zarr
│   │   ├── hires-sharp.zarr
│   │   ├── lores.zarr
│   │   └── sharp.zarr
│   ├── 64
│   │   ├── gauss.zarr
│   │   ├── hires-gauss.zarr
│   │   ├── hires-sharp.zarr
│   │   ├── lores.zarr
│   │   └── sharp.zarr
│   ├── 96
│   │   ├── gauss.zarr
│   │   ├── hires-gauss.zarr
│   │   ├── hires-sharp.zarr
│   │   ├── lores.zarr
│   │   └── sharp.zarr
│   └── hires.zarr
* `eddy/jet` -- configuration of the pyqg. eddy is default
* `hires.zarr` -- high-resolution simulation at 256x256 grid
* `48/64/96` -- resolution of the coarse models
* `lores.zarr` -- low-resolution simulation
* `gauss.zarr`, `sharp.zarr` -- training datasets for prediction of subgrid forcing obtained with Gaussian or Sharp filters
* `hires-gauss.zarr`, `hires-shapr.zarr` -- high-resolution simulation projected onto coarse grid with Gaussian or Sharp filters

In [ ]:


https://g-402b19.00888.8540.data.globus.org/eddy/48/gauss.zarr/lev/0